Librerías

In [30]:
! pip install gender_guesser
! pip install unidecode
! pip install bertopic
! pip install spacy gender-guesser unidecode
! python -m spacy download es_core_news_md
! pip install bertopic sentence-transformers
! python -m spacy download es_core_news_sm
! pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 28.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 75.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [31]:
import pandas as pd
import numpy as np
import spacy
import gender_guesser.detector as gender
import re
import unidecode
from collections import Counter
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import nltk
from nltk.corpus import stopwords

**Realizamos el procesamiento y limpieza de los datos.**

Cargamos y visualizamos el dataset para una primera inspección visual

In [32]:
df = pd.read_csv("peliculas_1940_2025.csv")
df

,id,title,release_date,year,genre,duration_minutes,plot
0,350295e9-aad9-4160-8e09-0d44221e32b6,¡Caigan las rosas blancas!,01/02/2025,2025,Drama,123.0,Violeta (Carolina Alamino) es una directora de...
1,f0b76aa5-3018-4279-b0dd-c8333b16ad6f,8,16/03/2025,2025,Drama histórico,126.0,8 es una película española de drama histórico ...
2,0ad46be2-3fcf-4bd8-9ed0-a48e79a64612,Almudena,15/03/2025,2025,Documental,80.0,La película se centra en celebrar la vida y ob...
3,a009e701-a94f-4834-bedf-a4dd5aa98f75,Antes de nós,20/05/2025,2025,Drama biográfico,84.0,La película se centra en dos momentos clave de...
4,e0bdeb7c-6423-4b99-a441-66ca63057360,Bajo un volcán,20/06/2025,2025,AventurasCatástrofesDrama romántico,92.0,Cuando la noticia de la posible erupción inmin...
...,...,...,...,...,...,...,...
3102,8f48fba1-98b7-4c4f-97ff-8ce7ed9fef5c,La Dolores,NaN,1940,Drama Musical,102.0,Una joven guapa y de una voz preciosa trabaja ...
3103,07b9848d-0c8e-4dd3-98b0-168be8847938,La florista de la reina,NaN,1940,Drama,89.0,Juan Manuel es un poeta que se traslada a Madr...
3104,c59abc4f-e099-4d4d-9d07-548911366375,La malquerida,NaN,1940,Drama,94.0,En la hacienda El Soto viven Doña Raimunda y s...
3105,782ca546-5e9c-472c-9339-a35cf171bc08,Marianela,NaN,1940,Drama,87.0,Marianela es una película dirigida por Benito ...


Analizamos dimensiones, tipos de datos y valores faltantes

In [33]:
print("Dimensiones del dataset:", df.shape)
print("\nNombres de columnas:")
print(df.columns.tolist())

# Revisar tipos de datos
print("\nInformación general del DataFrame:")
df.info()

# Ver valores únicos por columna clave
print("\nValores únicos por columna:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} únicos")

# Detectar valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

Dimensiones del dataset: (3107, 7)

Nombres de columnas:
['id', 'title', 'release_date', 'year', 'genre', 'duration_minutes', 'plot']

Información general del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3107 entries, 0 to 3106
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                3107 non-null   object 
 1   title             3107 non-null   object 
 2   release_date      287 non-null    object 
 3   year              3107 non-null   int64  
 4   genre             3096 non-null   object 
 5   duration_minutes  2849 non-null   float64
 6   plot              3107 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 170.0+ KB

Valores únicos por columna:
id: 3107 únicos
title: 3077 únicos
release_date: 247 únicos
year: 86 únicos
genre: 812 únicos
duration_minutes: 140 únicos
plot: 3107 únicos

Valores nulos por columna:
id                     0
title         

Tratamos valores nulos.

- La columna 'release_date' contiene la mayoría de valores nulos y nuestro análisis depende del año que está registrado en la columna 'year', por lo tanto, podemos eliminarla.
- En la columna 'genre' encontramos solo 11 filas con el valor faltante asi que podemos eliminarlas sin que nos afecte al estudio

In [34]:
# Eliminar la columna 'release_date'
df = df.drop(columns=['release_date'])

# Eliminar filas donde 'genre' sea nulo
df = df.dropna(subset=['genre'])

print("Columnas actuales:", df.columns.tolist())
print("Filas restantes tras eliminar nulos en 'genre':", df.shape[0])

Columnas actuales: ['id', 'title', 'year', 'genre', 'duration_minutes', 'plot']
Filas restantes tras eliminar nulos en 'genre': 3096


- En la columna 'duration_minutes' se han rellenado los valores con la media de su lustro correspondiente. Para ello y de cara al análisis que vamos a realizar hemos creado una nueva columna llamada 'lustro' que clasifica las películas en función de su año de estreno.

In [35]:
# Crear la columna 'lustro'
df['lustrum'] = (df['year'] // 5) * 5

# Calcular la media por lustro
mean_by_lustrum = df.groupby('lustrum')['duration_minutes'].mean()

# Rellenar los valores nulos con la media de su lustro
df['duration_minutes'] = df.apply(
    lambda row: mean_by_lustrum[row['lustrum']] if pd.isna(row['duration_minutes']) else row['duration_minutes'],
    axis=1)

# Redondear la duración a entero
df['duration_minutes'] = df['duration_minutes'].round(0).astype(int)

# Verificar si quedan nulos
print("Valores nulos restantes en duration_minutes:", df['duration_minutes'].isnull().sum())

Valores nulos restantes en duration_minutes: 0


Comprobación de duplicados

In [36]:
# Detectar duplicados exactos (filas idénticas)
duplicados_exactos = df.duplicated()
print(f"Duplicados exactos: {duplicados_exactos.sum()}")

df[duplicados_exactos].head()

Duplicados exactos: 0


,id,title,year,genre,duration_minutes,plot,lustrum


Vamos a procesar la columna 'genre'

In [37]:
print(df['genre'].unique())

print("Número de géneros únicos:", df['genre'].nunique())

print(df['genre'].value_counts())

['Drama' 'Drama histórico' 'Documental' 'Drama biográfico'
 'AventurasCatástrofesDrama romántico' 'Cortometraje y cine LGBT'
 'Comedia' 'AntologíaDramaFantasía' 'Drama romántico' 'Thriller'
 'DramaAcciónRomántica' 'Drama, Película de carretera, Coming-of-age'
 'Aventuras' 'AventurasDrama' 'Comedia dramática' 'DramaThriller'
 'AnimaciónAventuras' 'Terror y suspenso' 'CrimenThrillerDrama'
 'Comedia de época' 'ComediaAventuras'
 'AventurasComedia de terrorMisterio' 'Romance y drama'
 'AcciónAventurasTerrorThriller' 'AnimaciónComediaEspionajeAventuras'
 'Drama políticoDrama biográfico' 'Drama y cine autobiográfico'
 'Drama, cine postapocalíptico y película de carretera'
 'Drama psicológico' 'Comedia románticaMusical' 'documental'
 'Drama de época' 'TerrorSobrenatural' 'Comedia romántica'
 'ComediaDeportes' 'documental/ficción' 'AnimaciónAventurasFantasía'
 'ThrillerDramaAventuras' 'Terror, fantasía, drama y cine de época'
 'Terror' 'ThrillerRomance' 'ComediaSátira política'
 'Ciencia ficci

Encontramos muchos subgéneros que dificultan el análisis por eso vamos a mapearlos en clases más amplias que representen los géneros más comunes.

In [38]:
generos_principales = [
    "Drama",
    "Comedia",
    "Acción",
    "Aventura",
    "Ciencia ficción",
    "Fantasía",
    "Terror",
    "Suspenso",
    "Crimen",
    "Romance",
    "Animación",
    "Documental",
    "Musical",
    "Histórico",
    "Biográfico",
    "Bélico",
    "Western",
    "LGBT"
]

genre_mapping = {
    # Drama y derivados
    "drama": "Drama",
    "drama histórico": "Drama",
    "drama biográfico": "Drama",
    "drama romántico": "Drama",
    "melodrama": "Drama",

    # Comedia
    "comedia": "Comedia",
    "comedia romántica": "Comedia",
    "comedia dramática": "Comedia",
    "comedia negra": "Comedia",

    # Acción / Aventura
    "acción": "Acción",
    "aventuras": "Aventura",
    "aventurascatástrofesdrama romántico": "Aventura",
    "bélico": "Bélico",

    # Romance
    "romance": "Romance",
    "romántico": "Romance",

    # Terror / Suspenso / Crimen
    "terror": "Terror",
    "thriller": "Suspenso",
    "suspenso": "Suspenso",
    "crimen": "Crimen",
    "policíaco": "Crimen",

    # Ciencia ficción / Fantasía
    "ciencia ficción": "Ciencia ficción",
    "fantasía": "Fantasía",

    # Documental
    "documental": "Documental",

    # Animación
    "animación": "Animación",

    # Histórico / Biográfico
    "histórico": "Histórico",
    "biográfico": "Biográfico",

    # Musical
    "musical": "Musical",

    # Western
    "western": "Western",

    # LGBT
    "cortometraje y cine lgbt": "LGBT",
    "lgbt": "LGBT",
}


# Normalizar antes de mapear
df['genre_clean'] = (
    df['genre']
    .str.lower()
    .str.replace(r'[^a-záéíóúüñ\s]', '', regex=True)  # eliminar símbolos raros
    .str.strip()
)

# Aplicar el mapeo
df['genre_simplified'] = df['genre_clean'].map(genre_mapping).fillna("Otros")

# Ver distribución final
print(df['genre_simplified'].value_counts())

genre_simplified
Otros              1098
Drama               848
Comedia             768
Documental          132
Suspenso             60
Musical              52
Terror               51
Western              24
Animación            18
Aventura             18
Ciencia ficción       8
Fantasía              5
Acción                4
Romance               3
Crimen                2
Histórico             2
Bélico                2
LGBT                  1
Name: count, dtype: int64


Al observar que hay muchas películas que están clasificadas dentro del género 'Otros', vemos qué generos incluye esta categoría para, manualmente, introducirlos dentro de alguna otra.

In [39]:
# Filtrar las filas donde el género simplificado es 'Otros'
otros = df[df['genre_simplified'] == "Otros"]

# Ver los géneros originales únicos que cayeron en 'Otros'
print("Géneros originales clasificados como 'Otros':")
print(otros['genre_clean'].unique())

# Contar cuántas películas hay de cada uno
print("\nFrecuencia por género original dentro de 'Otros':")
print(otros['genre_clean'].value_counts())

Géneros originales clasificados como 'Otros':
['antologíadramafantasía' 'dramaacciónromántica'
 'drama película de carretera comingofage' 'aventurasdrama'
 'dramathriller' 'animaciónaventuras' 'terror y suspenso'
 'crimenthrillerdrama' 'comedia de época' 'comediaaventuras'
 'aventurascomedia de terrormisterio' 'romance y drama'
 'acciónaventurasterrorthriller' 'animacióncomediaespionajeaventuras'
 'drama políticodrama biográfico' 'drama y cine autobiográfico'
 'drama cine postapocalíptico y película de carretera' 'drama psicológico'
 'comedia románticamusical' 'drama de época' 'terrorsobrenatural'
 'comediadeportes' 'documentalficción' 'animaciónaventurasfantasía'
 'thrillerdramaaventuras' 'terror fantasía drama y cine de época'
 'thrillerromance' 'comediasátira política'
 'ciencia ficciónthriller político' 'wésterndrama de época'
 'drama históricodrama psicológico' 'ficción'
 'comedia romántica y cine lgbt' 'comedia musical' 'thrillerdrama'
 'comedia dramática de carretera' 'drama mus

In [40]:
# Primero, asegurarnos de que la columna esté en minúsculas
df['genre_clean'] = df['genre'].str.lower()

# Crear la columna simplificada usando el mapeo base
df['genre_simplified'] = df['genre_clean'].map(genre_mapping)

# Luego, reemplazar cualquier valor que contenga 'drama' por 'Drama'
df.loc[df['genre_clean'].str.contains('drama|dramático|cine quinqui', na=False), 'genre_simplified'] = 'Drama'

# Los valores aún nulos o no mapeados se pueden clasificar como 'Otros'
df['genre_simplified'] = df['genre_simplified'].fillna('Otros')

# Ver distribución final
print(df['genre_simplified'].value_counts())

genre_simplified
Drama              1319
Comedia             767
Otros               629
Documental          132
Suspenso             60
Musical              52
Terror               51
Western              24
Animación            18
Aventura             17
Ciencia ficción       8
Fantasía              5
Acción                4
Romance               3
Crimen                2
Histórico             2
Bélico                2
LGBT                  1
Name: count, dtype: int64


Realizamos el mismo paso que hemos hecho con las películas que contenían la palabra 'drama' pero esta vez con el resto de géneros.

In [41]:
# Filtrar solo los que actualmente están en 'Otros' y contienen 'comedia'
mask_comedia = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains('comedia', na=False)
# Asignar 'Comedia' a esos géneros
df.loc[mask_comedia, 'genre_simplified'] = 'Comedia'


# Filtrar los géneros en 'Otros' que contengan cualquiera de esas palabras clave
mask_suspenso = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'thriller|suspense|suspenso|misterio|intriga|crímenes',
    case=False,
    na=False)
# Asignar 'Suspenso' a esos géneros
df.loc[mask_suspenso, 'genre_simplified'] = 'Suspenso'


mask_terror = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains('terror', na=False)
df.loc[mask_terror, 'genre_simplified'] = 'Terror'


mask_western = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'wéstern|western',
    case=False,na=False)
df.loc[mask_western, 'genre_simplified'] = 'Western'


mask_ficcion = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'ficción|ficcion|ciencia|fantástico',
    case=False,
    na=False)
df.loc[mask_ficcion, 'genre_simplified'] = 'Ciencia ficción'


mask_accion = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'espías|espionaje|policiaco|policíaco|policiaca|acción|accion|Acción|policial|atracos',
    case=False,
    na=False)
df.loc[mask_accion, 'genre_simplified'] = 'Acción'


mask_docu = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'documental|docu|documentalcine',
    case=False,
    na=False)
df.loc[mask_docu, 'genre_simplified'] = 'Documental'


mask_hist = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'biografía|histórico|historico|biografia|históricas|biográfica|histórica|época',
    case=False,
    na=False)
df.loc[mask_hist, 'genre_simplified'] = 'Histórico'


mask_aventura = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'aventuras|aventura|aventurafantástico|animaciónaventuras|animaciónaventurasfantasía|peplum',
    case=False,
    na=False)
df.loc[mask_aventura, 'genre_simplified'] = 'Aventura'


mask_musical = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'musical',
    case=False,
    na=False)
df.loc[mask_musical, 'genre_simplified'] = 'Musical'


mask_animacion = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'animación|infantil|familiar|familia',
    case=False,
    na=False)
df.loc[mask_animacion, 'genre_simplified'] = 'Animación'


mask_belico = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'bélico|bélica|bélica/propagandística',
    case=False,
    na=False)
df.loc[mask_belico, 'genre_simplified'] = 'Bélico'


mask_lgbt = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'lésbico|lgbt',
    case=False,
    na=False)
df.loc[mask_lgbt, 'genre_simplified'] = 'LGBT'


mask_erotico = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'erótico|adultos|erótica',
    case=False,
    na=False)
df.loc[mask_erotico, 'genre_simplified'] = 'Erótico'


mask_romance = (df['genre_simplified'] == 'Otros') & df['genre_clean'].str.contains(
    'romántica',
    case=False,
    na=False)
df.loc[mask_romance, 'genre_simplified'] = 'Romance'

In [42]:
# Ver cuántos quedan aún como 'Otros'
print(df['genre_simplified'].value_counts())

genre_simplified
Drama              1319
Comedia            1012
Suspenso            174
Documental          148
Terror               81
Western              74
Musical              57
Acción               41
Histórico            39
Ciencia ficción      37
Aventura             33
Animación            27
Otros                25
Bélico                7
Erótico               6
Romance               6
Fantasía              5
LGBT                  3
Crimen                2
Name: count, dtype: int64


In [43]:
# Filtrar los géneros actualmente en 'Otros'
otros_actuales = df[df['genre_simplified'] == 'Otros']

# Ver los géneros originales únicos que cayeron en 'Otros'
print("Géneros originales aún en 'Otros':")
print(otros_actuales['genre_clean'].unique())

# Ver frecuencia de cada uno
print("\nFrecuencia de géneros en 'Otros':")
print(otros_actuales['genre_clean'].value_counts())

Géneros originales aún en 'Otros':
['humor surrealista, poesía' 'cortometraje'
 'romancecolonialismo años 1950 y 1960.' 'biopic' 'mónsteresépica'
 'cine de naturaleza' 'humor absurdo' 'tv movie'
 'tv movie, miniserie, adolescente' 'cine negro'
 'cine de autor / realismo mágico' 'tvmovie' 'serie b/fantástica'
 'sramasocial' 'humor negro' 'corto' 'cine de vampiros' 'religiosa'
 'largometraje' 'multigenérica']

Frecuencia de géneros en 'Otros':
genre_clean
cortometraje                             3
biopic                                   2
cine negro                               2
tv movie                                 2
romancecolonialismo años 1950 y 1960.    1
humor surrealista, poesía                1
cine de naturaleza                       1
mónsteresépica                           1
humor absurdo                            1
tv movie, miniserie, adolescente         1
cine de autor / realismo mágico          1
tvmovie                                  1
serie b/fantástica        

**Creamos indicadores mínimos y analizamos sus resultados**

Volumen temporal de publicaciones: conteo por lustro

In [62]:
# Calcular el conteo por lustro
lustrum_counts = df["lustrum"].value_counts().to_dict()

# Crear la nueva columna con el conteo correspondiente
df["count_by_lustrum"] = df["lustrum"].map(lustrum_counts)

df[["title", "lustrum", "count_by_lustrum"]].head()


,title,lustrum,count_by_lustrum
0,¡Caigan las rosas blancas!,2025,64
1,8,2025,64
2,Almudena,2025,64
3,Antes de nós,2025,64
4,Bajo un volcán,2025,64


Para obtener el sentimiento y los tópicos de las sinopsis es necesario realizar un preprocesamiento para eliminar articulos y palabras comunes (stopwords), caracteres no alfabéticos, ...

In [45]:
# Pasar todo a minúsculas
df['plot_clean'] = df['plot'].str.lower()

# Eliminar caracteres no alfabéticos
df['plot_clean'] = df['plot_clean'].apply(lambda x: re.sub(r'[^a-záéíóúüñ\s]', '', str(x)))

# Eliminar espacios repetidos
df['plot_clean'] = df['plot_clean'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Eliminar stopwords
nltk.download('stopwords')
stopwords_es = set(stopwords.words('spanish'))

df['plot_clean'] = df['plot_clean'].apply(
    lambda x: ' '.join([w for w in str(x).split() if w not in stopwords_es])
)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Análisis sentimiento sinopsis

In [46]:
# TextBlob
df['sentiment_textblob'] = df['plot'].apply(lambda s: TextBlob(s).sentiment.polarity)
df.head()

,id,title,year,genre,duration_minutes,plot,lustrum,genre_clean,genre_simplified,count_by_lustrum,plot_clean,sentiment_textblob
0,350295e9-aad9-4160-8e09-0d44221e32b6,¡Caigan las rosas blancas!,2025,Drama,123,Violeta (Carolina Alamino) es una directora de...,2025,drama,Drama,64,violeta carolina alamino directora cine contra...,-0.25
1,f0b76aa5-3018-4279-b0dd-c8333b16ad6f,8,2025,Drama histórico,126,8 es una película española de drama histórico ...,2025,drama histórico,Drama,64,película española drama histórico escrita copr...,0.00
2,0ad46be2-3fcf-4bd8-9ed0-a48e79a64612,Almudena,2025,Documental,80,La película se centra en celebrar la vida y ob...,2025,documental,Documental,64,película centra celebrar vida obra fallecida e...,0.00
3,a009e701-a94f-4834-bedf-a4dd5aa98f75,Antes de nós,2025,Drama biográfico,84,La película se centra en dos momentos clave de...,2025,drama biográfico,Drama,64,película centra dos momentos clave relación ca...,0.00
4,e0bdeb7c-6423-4b99-a441-66ca63057360,Bajo un volcán,2025,AventurasCatástrofesDrama romántico,92,Cuando la noticia de la posible erupción inmin...,2025,aventurascatástrofesdrama romántico,Drama,64,noticia posible erupción inminente volcán tene...,0.10


In [47]:
num_ceros = (df['sentiment_textblob'] == 0).sum()
print(f"Número de valores con sentimiento exactamente 0: {num_ceros}")

Número de valores con sentimiento exactamente 0: 2587


Observamos que la mayoría de sinopsis muestran un sentimiento neutro, lo que no nos aporta información relevante.
Vamos a probar otros dos modelos y calcular la media para ver si es posible sacar alguna idea concluyente.

In [48]:
# Pysentimiento Roberta (español)
pipe1 = pipeline("text-classification", model="pysentimiento/robertuito-sentiment-analysis", truncation=True)
df['sentiment_robertuito'] = df['plot'].apply(lambda x: pipe1(x)[0]['label'])

df.head()

Device set to use cuda:0


,id,title,year,genre,duration_minutes,plot,lustrum,genre_clean,genre_simplified,count_by_lustrum,plot_clean,sentiment_textblob,sentiment_robertuito
0,350295e9-aad9-4160-8e09-0d44221e32b6,¡Caigan las rosas blancas!,2025,Drama,123,Violeta (Carolina Alamino) es una directora de...,2025,drama,Drama,64,violeta carolina alamino directora cine contra...,-0.25,NEG
1,f0b76aa5-3018-4279-b0dd-c8333b16ad6f,8,2025,Drama histórico,126,8 es una película española de drama histórico ...,2025,drama histórico,Drama,64,película española drama histórico escrita copr...,0.00,POS
2,0ad46be2-3fcf-4bd8-9ed0-a48e79a64612,Almudena,2025,Documental,80,La película se centra en celebrar la vida y ob...,2025,documental,Documental,64,película centra celebrar vida obra fallecida e...,0.00,NEG
3,a009e701-a94f-4834-bedf-a4dd5aa98f75,Antes de nós,2025,Drama biográfico,84,La película se centra en dos momentos clave de...,2025,drama biográfico,Drama,64,película centra dos momentos clave relación ca...,0.00,NEU
4,e0bdeb7c-6423-4b99-a441-66ca63057360,Bajo un volcán,2025,AventurasCatástrofesDrama romántico,92,Cuando la noticia de la posible erupción inmin...,2025,aventurascatástrofesdrama romántico,Drama,64,noticia posible erupción inminente volcán tene...,0.10,NEU


In [49]:
# Multilingüe BERT 1–5 estrellas
pipe2 = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment", truncation=True)
df['sentiment_bertstars'] = df['plot'].apply(lambda x: pipe2(x)[0]['label'])

df.head()

Device set to use cuda:0


,id,title,year,genre,duration_minutes,plot,lustrum,genre_clean,genre_simplified,count_by_lustrum,plot_clean,sentiment_textblob,sentiment_robertuito,sentiment_bertstars
0,350295e9-aad9-4160-8e09-0d44221e32b6,¡Caigan las rosas blancas!,2025,Drama,123,Violeta (Carolina Alamino) es una directora de...,2025,drama,Drama,64,violeta carolina alamino directora cine contra...,-0.25,NEG,3 stars
1,f0b76aa5-3018-4279-b0dd-c8333b16ad6f,8,2025,Drama histórico,126,8 es una película española de drama histórico ...,2025,drama histórico,Drama,64,película española drama histórico escrita copr...,0.00,POS,4 stars
2,0ad46be2-3fcf-4bd8-9ed0-a48e79a64612,Almudena,2025,Documental,80,La película se centra en celebrar la vida y ob...,2025,documental,Documental,64,película centra celebrar vida obra fallecida e...,0.00,NEG,4 stars
3,a009e701-a94f-4834-bedf-a4dd5aa98f75,Antes de nós,2025,Drama biográfico,84,La película se centra en dos momentos clave de...,2025,drama biográfico,Drama,64,película centra dos momentos clave relación ca...,0.00,NEU,4 stars
4,e0bdeb7c-6423-4b99-a441-66ca63057360,Bajo un volcán,2025,AventurasCatástrofesDrama romántico,92,Cuando la noticia de la posible erupción inmin...,2025,aventurascatástrofesdrama romántico,Drama,64,noticia posible erupción inminente volcán tene...,0.10,NEU,4 stars


Para calcular la media tenemos que convertir los tres sentimientos a una categoría común:
- TextBlob → negativo si < 0, neutro si = 0 y positivo si > 0
- Robertuito ya está listo (NEG, NEU, POS)
- BertStars → 1-2 stars = NEG, 3 stars = NEU y 4-5 stars = POS

A continuación contamos las categorías y elegimos la que aparece más veces guardando el resultado en la columna 'media_sentiment'.

In [50]:
# TextBlob → Etiquetas
def textblob_to_label(val):
    if val > 0:
        return "POS"
    elif val < 0:
        return "NEG"
    else:
        return "NEU"

df["sentiment_textblob_label"] = df["sentiment_textblob"].apply(textblob_to_label)

# BertStars → Etiquetas
def stars_to_label(stars):
    try:
        num = int(str(stars).split()[0])
        if num <= 2:
            return "NEG"
        elif num == 3:
            return "NEU"
        else:
            return "POS"
    except:
        return "NEU"  # por si hay valores nulos o raros

df["sentiment_bert_label"] = df["sentiment_bertstars"].apply(stars_to_label)

# Calcular el sentimiento mayoritario
def majority_sentiment(row):
    vals = [row["sentiment_textblob_label"], row["sentiment_robertuito"], row["sentiment_bert_label"]]
    # Contar frecuencia de cada valor y elegir el más común
    return max(set(vals), key=vals.count)

df["media_sentiment"] = df.apply(majority_sentiment, axis=1)

# Resultado final
df[["sentiment_textblob", "sentiment_robertuito", "sentiment_bertstars", "media_sentiment"]].head()


,sentiment_textblob,sentiment_robertuito,sentiment_bertstars,media_sentiment
0,-0.25,NEG,3 stars,NEG
1,0.00,POS,4 stars,POS
2,0.00,NEG,4 stars,NEG
3,0.00,NEU,4 stars,NEU
4,0.10,NEU,4 stars,POS


In [51]:
neu_count = (df["media_sentiment"] == "NEU").sum()
print("Cantidad de NEU:", neu_count)

Cantidad de NEU: 1547


La mayoría de sinopsis siguen dando neutras depués de realizar la media de los tres modelos, por tanto no podemos concluir nada.

Distribución de tópicos por intervalo de tiempo

In [52]:
# Lista para guardar los resultados por lustro
topics_summary = []

# Iterar sobre cada grupo temporal
for lustrum, group in df.groupby('lustrum'):
    print(f"🔹 Procesando lustro {lustrum} con {len(group)} películas...")

    # Entrenar un modelo BERTopic SOLO con los textos de ese lustro
    topic_model = BERTopic(language="spanish", verbose=False)
    topics, probs = topic_model.fit_transform(group['plot_clean'])

    # Obtener los tópicos generados y sus palabras clave
    topic_info = topic_model.get_topic_info()
    topic_info = topic_info[topic_info['Topic'] != -1]  # excluir ruido

    # Generar una lista con "Tópico X: palabra1, palabra2, palabra3"
    topic_list = []
    for _, row in topic_info.iterrows():
        topic_words = topic_model.get_topic(row['Topic'])
        keywords = ', '.join([w for w, _ in topic_words])
        topic_list.append(f"Tópico {row['Topic']}: {keywords}")

    # Unir todo en un solo string
    topic_summary = ' | '.join(topic_list)

    # Guardar el resumen por lustro
    topics_summary.append({'lustrum': lustrum, 'topics_lustrum_keywords': topic_summary})

# Convertir a DataFrame
df_topics_summary = pd.DataFrame(topics_summary)

df = df.merge(df_topics_summary, on='lustrum', how='left')

# Verificar resultado
df[['title', 'lustrum', 'topics_lustrum_keywords']].head()


🔹 Procesando lustro 1940 con 70 películas...
🔹 Procesando lustro 1945 con 83 películas...
🔹 Procesando lustro 1950 con 123 películas...
🔹 Procesando lustro 1955 con 136 películas...
🔹 Procesando lustro 1960 con 218 películas...
🔹 Procesando lustro 1965 con 230 películas...
🔹 Procesando lustro 1970 con 200 películas...
🔹 Procesando lustro 1975 con 173 películas...
🔹 Procesando lustro 1980 con 173 películas...
🔹 Procesando lustro 1985 con 138 películas...
🔹 Procesando lustro 1990 con 102 películas...
🔹 Procesando lustro 1995 con 149 películas...
🔹 Procesando lustro 2000 con 230 películas...
🔹 Procesando lustro 2005 con 304 películas...
🔹 Procesando lustro 2010 con 244 películas...
🔹 Procesando lustro 2015 con 220 películas...
🔹 Procesando lustro 2020 con 239 películas...
🔹 Procesando lustro 2025 con 64 películas...


,title,lustrum,topics_lustrum_keywords
0,¡Caigan las rosas blancas!,2025,"Tópico 0: familia, joven, guerra, ven, vida, d..."
1,8,2025,"Tópico 0: familia, joven, guerra, ven, vida, d..."
2,Almudena,2025,"Tópico 0: familia, joven, guerra, ven, vida, d..."
3,Antes de nós,2025,"Tópico 0: familia, joven, guerra, ven, vida, d..."
4,Bajo un volcán,2025,"Tópico 0: familia, joven, guerra, ven, vida, d..."


In [63]:
# Imprimir solo la información resumida por lustro
print("\n📊 Tópicos principales por lustro:\n")
print(df_topics_summary[['lustrum', 'topics_lustrum_keywords']].to_string(index=False))


📊 Tópicos principales por lustro:

 lustrum                                                                                                                                                                                                                                                                                                                                                                                                                                            topics_lustrum_keywords
    1940                                                                                                                                                                                                                                                                                          Tópico 0: joven, hijo, situación, decide, áurea, acacia, juan, casa, hija, vida | Tópico 1: española, película, dirigida, gil, pedro, guerra, rafael, basada, tony, films
    1945                                    

Tampoco obtenemos tópicos de gran interés.
Ambos indicadores quedaran descartados en nuestro estudio.





**Indicador específico de estudio**

Al plantear una hipótesis relacionada con la presencia femenina en las películas, que lo vemos reflejado en la presencia en la sinopsis, calculamos varios indicadores que nos puedan resultar útiles.
- 'female_presence' es una representación binaria. 1: hay presencia femenina en la sinopsis ya sea por términos (chica. mujer, ...) o por instancias de personas. 0: en el caso contrario.
- 'count_female' y 'count_male' muestra el número de veces que aparece en cada sinopsis una instancia o referencia a cada uno de los generos.
- 'female_ratio' pondera la cuenta femenina con respecto a la total (male + female)

In [54]:
# Cargar modelo y detector
nlp = spacy.load("es_core_news_md")
d = gender.Detector()

# Palabras genéricas
female_terms = [
    "mujer", "chica", "niña", "madre", "hermana", "hija",
    "esposa", "novia", "femenina", "reina", "princesa",
    "ella", "heroína"
]

male_terms = [
    "hombre", "chico", "niño", "padre", "hermano", "hijo",
    "esposo", "novio", "masculino", "rey", "príncipe",
    "él", "héroe"
]

# Función principal
def analyze_gender_mentions(text):
    """Devuelve conteos y presencia femenina en un texto."""
    if pd.isna(text):
        return pd.Series([0, 0, 0])  # female_count, male_count, female_presence

    # Normalizar texto
    text = unidecode.unidecode(text.lower())
    doc = nlp(text)

    female_count = 0
    male_count = 0

    # Nombres propios detectados por spaCy + gender_guesser
    for ent in doc.ents:
        if ent.label_ == "PER":
            first_name = ent.text.split()[0].capitalize()
            gender_pred = d.get_gender(first_name)
            if gender_pred in ["female", "mostly_female"]:
                female_count += 1
            elif gender_pred in ["male", "mostly_male"]:
                male_count += 1

    # Palabras genéricas
    female_count += sum(len(re.findall(rf"\b{term}\b", text)) for term in female_terms)
    male_count += sum(len(re.findall(rf"\b{term}\b", text)) for term in male_terms)

    # Presencia femenina (1 si hay al menos una mención)
    female_presence = 1 if female_count > 0 else 0

    return pd.Series([female_count, male_count, female_presence])

# Aplicar al DataFrame
df[["count_female", "count_male", "female_presence"]] = df["plot"].apply(analyze_gender_mentions)

# Calcular proporción de presencia femenina
df["female_ratio"] = df.apply(
    lambda row: row["count_female"] / (row["count_female"] + row["count_male"])
    if (row["count_female"] + row["count_male"]) > 0 else 0,
    axis=1
)

# Resultado
df[["count_female", "count_male", "female_presence", "female_ratio"]].head()


,count_female,count_male,female_presence,female_ratio
0,3,0,1,1.000000
1,1,2,1,0.333333
2,1,1,1,0.500000
3,0,1,0,0.000000
4,1,1,1,0.500000


Nuevas columnas:
- Longuitud de la sinopsis
- Adjetivos femeninos

In [55]:
# Crear la columna 'word_count' con el número de palabras de la sinopsis
df['word_plot_count'] = df['plot'].apply(
    lambda x: len(str(x).split()) if pd.notnull(x) else 0
)

df[['title', 'word_plot_count']].head()


,title,word_plot_count
0,¡Caigan las rosas blancas!,66
1,8,55
2,Almudena,46
3,Antes de nós,84
4,Bajo un volcán,63


In [ ]:
# Crear la columna 'female_adj_percentage' con el porcentaje de adjetivos femeninos.
# Cargar el modelo de idioma español de spaCy.
nlp = spacy.load("es_core_news_md")

def female_adjectives_percentage(text):
    if not isinstance(text, str) or text.strip() == "":
        return 0.0
    doc = nlp(text)
    total, fem = 0, 0
    for token in doc:
        if token.pos_ == "ADJ":
            total += 1
            if token.morph.get("Gender") == ["Fem"]:
                fem += 1
    return (fem / total * 100) if total > 0 else 0.0

In [57]:
df["female_adj_percentage"] = df["plot"].apply(female_adjectives_percentage)


In [58]:
df.head()

,id,title,year,genre,duration_minutes,plot,lustrum,genre_clean,genre_simplified,count_by_lustrum,...,sentiment_textblob_label,sentiment_bert_label,media_sentiment,topics_lustrum_keywords,count_female,count_male,female_presence,female_ratio,word_plot_count,female_adj_percentage
0,350295e9-aad9-4160-8e09-0d44221e32b6,¡Caigan las rosas blancas!,2025,Drama,123,Violeta (Carolina Alamino) es una directora de...,2025,drama,Drama,64,...,NEG,NEU,NEG,"Tópico 0: familia, joven, guerra, ven, vida, d...",3,0,1,1.000000,66,71.428571
1,f0b76aa5-3018-4279-b0dd-c8333b16ad6f,8,2025,Drama histórico,126,8 es una película española de drama histórico ...,2025,drama histórico,Drama,64,...,NEU,POS,POS,"Tópico 0: familia, joven, guerra, ven, vida, d...",1,2,1,0.333333,55,71.428571
2,0ad46be2-3fcf-4bd8-9ed0-a48e79a64612,Almudena,2025,Documental,80,La película se centra en celebrar la vida y ob...,2025,documental,Documental,64,...,NEU,POS,NEG,"Tópico 0: familia, joven, guerra, ven, vida, d...",1,1,1,0.500000,46,33.333333
3,a009e701-a94f-4834-bedf-a4dd5aa98f75,Antes de nós,2025,Drama biográfico,84,La película se centra en dos momentos clave de...,2025,drama biográfico,Drama,64,...,NEU,POS,NEU,"Tópico 0: familia, joven, guerra, ven, vida, d...",0,1,0,0.000000,84,50.000000
4,e0bdeb7c-6423-4b99-a441-66ca63057360,Bajo un volcán,2025,AventurasCatástrofesDrama romántico,92,Cuando la noticia de la posible erupción inmin...,2025,aventurascatástrofesdrama romántico,Drama,64,...,POS,POS,POS,"Tópico 0: familia, joven, guerra, ven, vida, d...",1,1,1,0.500000,63,16.666667


**df final**

In [59]:
df.columns

Index(['id', 'title', 'year', 'genre', 'duration_minutes', 'plot', 'lustrum',
       'genre_clean', 'genre_simplified', 'count_by_lustrum', 'plot_clean',
       'sentiment_textblob', 'sentiment_robertuito', 'sentiment_bertstars',
       'sentiment_textblob_label', 'sentiment_bert_label', 'media_sentiment',
       'topics_lustrum_keywords', 'count_female', 'count_male',
       'female_presence', 'female_ratio', 'word_plot_count',
       'female_adj_percentage'],
      dtype='object')

Eliminamos las columnas de las que ya hemos obtenido información o aquellas que no nos aportan datos de interés para nuestro caso de estudio (sí se mantiene las columnas referentes a tópicos y sentimiento por si en un futuro se quiere utilizar estas medidas para profundizar en la hipótesis).

In [60]:
# Mantener las columnas que nos interesan
columns = [
    'id', 'year','lustrum','count_by_lustrum','duration_minutes',
    'genre', 'genre_simplified','word_plot_count',
    'media_sentiment', 'topics_lustrum_keywords',
    'count_female', 'count_male', 'female_presence',
    'female_ratio', 'female_adj_percentage'
]

df_final = df[columns]


# Guardar el DataFrame limpio en un archivo CSV
df_final.to_csv("df_entrega_2.csv", index=False)

print("Columnas actuales:", df_final.columns.tolist())


Columnas actuales: ['id', 'year', 'lustrum', 'count_by_lustrum', 'duration_minutes', 'genre', 'genre_simplified', 'word_plot_count', 'media_sentiment', 'topics_lustrum_keywords', 'count_female', 'count_male', 'female_presence', 'female_ratio', 'female_adj_percentage']


In [64]:
df_final.head()

,id,year,lustrum,count_by_lustrum,duration_minutes,genre,genre_simplified,word_plot_count,media_sentiment,topics_lustrum_keywords,count_female,count_male,female_presence,female_ratio,female_adj_percentage
0,350295e9-aad9-4160-8e09-0d44221e32b6,2025,2025,64,123,Drama,Drama,66,NEG,"Tópico 0: familia, joven, guerra, ven, vida, d...",3,0,1,1.000000,71.428571
1,f0b76aa5-3018-4279-b0dd-c8333b16ad6f,2025,2025,64,126,Drama histórico,Drama,55,POS,"Tópico 0: familia, joven, guerra, ven, vida, d...",1,2,1,0.333333,71.428571
2,0ad46be2-3fcf-4bd8-9ed0-a48e79a64612,2025,2025,64,80,Documental,Documental,46,NEG,"Tópico 0: familia, joven, guerra, ven, vida, d...",1,1,1,0.500000,33.333333
3,a009e701-a94f-4834-bedf-a4dd5aa98f75,2025,2025,64,84,Drama biográfico,Drama,84,NEU,"Tópico 0: familia, joven, guerra, ven, vida, d...",0,1,0,0.000000,50.000000
4,e0bdeb7c-6423-4b99-a441-66ca63057360,2025,2025,64,92,AventurasCatástrofesDrama romántico,Drama,63,POS,"Tópico 0: familia, joven, guerra, ven, vida, d...",1,1,1,0.500000,16.666667


# Hipótesis
**La presencia de personajes principales femeninos ha aumentado a lo largo de los años**